In [ ]:
# !pip install mediapipe
# %pip install msvc-runtime
# !conda install tensorflow

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... failed


warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE

LibMambaUnsatisfiableError: Encountered problems while solving:
  - nothing provides bleach 1.5.0 needed by tensorboard-1.7.0-py35he025d50_1

Could not solve for environment specs
The following packages are incompatible
├─ pin-1 is installable and it requires
│  └─ python 3.12.* , which can be installed;
└─ tensorflow is not installable because there are no viable options
   ├─ tensorflow [1.10.0|1.9.0] would require
   │  └─ python 3.5.* , which conflicts with any installable versions previously reported;
   ├─ tensorflow [1.10.0|1.11.0|...|2.1.0] would require
   │  └─ python 3.6.* , which conflicts with any installable versions previously reported;
   ├─ tensorflow [1.13.1|1.14.0|...|2.9.1] would require
   │  └─ python 3.7.* , which conflicts with any installable versions previously reported;
   ├─ tensorflow [1.7.0|1.7.1|1.8.0] would require
   │  └─ tensorboard [>=1.7.0,<1.8.0 |>=1.8.0,<1.9.0 ], which r

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time



Là on fait une implementation direct basée sur mediapipe pour la detection d'objets pour une image, et on applique ça dans une boucle pour chaque frame de notre camera stream. Les modeles de détection d'objets contiennet d'ailleurs un classificateurs d'objets, avec une liste un peu limitée. 

In [ ]:

base_options = python.BaseOptions(model_asset_path='models/efficientdet_lite0_float16.tflite') #meilleure rapprot
# base_options = python.BaseOptions(model_asset_path='models/efficientdet_lite0_int8.tflite')
# base_options = python.BaseOptions(model_asset_path='models/efficientdet_lite0_float32.tflite')
# base_options = python.BaseOptions(model_asset_path='models/ssd_mobilenet_v2.tflite')
options = vision.ObjectDetectorOptions(base_options=base_options, max_results=4, score_threshold=0.5)
detector = vision.ObjectDetector.create_from_options(options)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    detection_result = detector.detect(image)

    annotated_frame = frame.copy()
    if detection_result.detections:
        for detection in detection_result.detections:
          
            bbox = detection.bounding_box
            x, y, w, h = bbox.origin_x, bbox.origin_y, bbox.width, bbox.height

            cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (100, 255, 0), 2)
            label = detection.categories[0].category_name if detection.categories else 'Object'
            cv2.putText(annotated_frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Real-time Object Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [11]:
detection_result.detections

[Detection(bounding_box=BoundingBox(origin_x=4, origin_y=40, width=630, height=435), categories=[Category(index=None, score=0.8906965255737305, display_name=None, category_name='person')], keypoints=[])]

### Using LIVE_STREAM 

Dans cette experimentation on utilise l'option LIVE_STREAM comme running_mode pour le detector_options, qui gère normalement plus fluidement les detection en live, et qui nécissite d'ailleurs un callback ainsi qu'un detecteur asynchrone qui depends des laps de temps des frames.

In [3]:
from utils import print_detection


In [ ]:
import argparse




def run(model: str) -> None:
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        sys.exit("ERREUR : Impossible d'ouvrir la webcam.")

    detection_results = []
    def result_callback(result: vision.ObjectDetectorResult,
                        output_image: mp.Image, timestamp_ms: int):
        result.timestamp_ms = timestamp_ms
        detection_results.append(result)

    base_options = python.BaseOptions(model_asset_path=model)
    detector_options = vision.ObjectDetectorOptions(
        base_options=base_options,
        running_mode=vision.RunningMode.LIVE_STREAM,
        score_threshold=0.5,
        max_results=5,
        result_callback=result_callback
    )
    detector = vision.ObjectDetector.create_from_options(detector_options)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            sys.exit("ERREUR : Impossible de lire depuis la webcam.")

        frame = cv2.flip(frame, 1)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

        detector.detect_async(mp_image, int(time.time() * 1000))

        output_image = mp_image.numpy_view()
        output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

        if detection_results:
            annotated_image = print_detection(output_image, detection_results[0])
            cv2.imshow("Object Detector", annotated_image)
            detection_results.clear()
        else:
            cv2.imshow("Object Detector", output_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    detector.close()
    cap.release()
    cv2.destroyAllWindows()



parser = argparse.ArgumentParser(
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument(
    '--model',
    help="Chemin du modèle (TFLite de détection d'objets)",
    default='models/efficientdet_lite0_int8.tflite')
args = parser.parse_args()

run(args.model)



### Implémentation d'un classificateur a part

La on utilise directement l'interpréteur TFLite pour la classification. Dans cette approche, le modèle de classification est chargé manuellement, les tenseurs sont gérés explicitement et un fichier de labels est utilisé pour mapper l'indice prédit à une étiquette. Cela offre un contrôle précis sur le prétraitement (redimensionnement, conversion de type, normalisation éventuelle) et sur le post-traitement, mais demande plus de code et peut entraîner des erreurs si l'ordre des labels n'est pas aligné avec les sorties du modèle, ce qui était malheureusement le cas pour moi.

In [ ]:

import tensorflow as tf
import sys

# -------------------------
# Chargement du classificateur TFLite
# -------------------------

def load_labels(label_path: str) -> dict:
    labels = {}
    with open(label_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            labels[i] = line.strip()
    return labels

LABELS = load_labels("labels/effiscientnet_lite0_labels_classifier.txt")
classifier_path = "models/efficientnet_lite0_float32_classifier.tflite"
interpreter = tf.lite.Interpreter(model_path=classifier_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def classify_crop(image_crop: np.ndarray) -> str:
    
    input_shape = input_details[0]['shape']  #[1, height, width, channels]
    target_height, target_width = input_shape[1], input_shape[2]
    resized = cv2.resize(image_crop, (target_width, target_height))
    input_data = np.expand_dims(resized, axis=0).astype(np.uint8)
    
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    pred_index = int(np.argmax(output_data))
    return LABELS.get(pred_index, "inconnu")

# -------------------------
# Configuration du détecteur d'objets (MediaPipe Tasks)
# -------------------------
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

base_options = python.BaseOptions(model_asset_path="models/efficientdet_lite0_float16.tflite")
detector_options = vision.ObjectDetectorOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.IMAGE,  # mode image pour un traitement synchrone
    score_threshold=0.5,
    max_results=5
)
detector = vision.ObjectDetector.create_from_options(detector_options)

# -------------------------
# Capture vidéo et traitement
# -------------------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    sys.exit("Erreur : impossible d'ouvrir la webcam.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
    
    detection_result = detector.detect(mp_image)
    for detection in detection_result.detections:
        bbox = detection.bounding_box
        x_min = bbox.origin_x
        y_min = bbox.origin_y
        x_max = x_min + bbox.width
        y_max = y_min + bbox.height

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        crop = frame[y_min:y_max, x_min:x_max]
        if crop.size == 0:
            continue  # Sauter si la ROI est invalide
        label = classify_crop(crop)
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    cv2.imshow("Detection & Classification", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Cette fois on exploite l'API ImageClassifier de MediaPipe Tasks directement pour la classification. Cette méthode simplifie grandement le pipeline car elle intègre directement dans le modèle (via les métadonnées) l'information sur les étiquettes, renvoyant ainsi le nom de la catégorie et le score sans nécessiter de gestion manuelle des labels. En gros, elle est plus simple, mais génère un peu d'erreurs de prédiction, car deja la façon dont on fait la classification sur le bounding box et le préprocessing peut entrainer des erreurs, en plus, ce classifier contient pluesieurs classes par rapport au classificateur integré de detection d'objets, et elle contient pas exactement les memes classes que l'autre modele aussi.

In [ ]:



# --- Détection d'objets (EfficientDet par exemple) ---
base_options_det = python.BaseOptions(model_asset_path="models/efficientdet_lite0_float16.tflite")
detector_options = vision.ObjectDetectorOptions(
    base_options=base_options_det,
    running_mode=vision.RunningMode.IMAGE,
    score_threshold=0.5,
    max_results=5
)
detector = vision.ObjectDetector.create_from_options(detector_options)

# --- Classification avec ImageClassifier ---
base_options_cls = python.BaseOptions(model_asset_path="models/efficientnet_lite0_int8_classifier.tflite")
classifier_options = vision.ImageClassifierOptions(
    base_options=base_options_cls,
    max_results=4  
)
classifier = vision.ImageClassifier.create_from_options(classifier_options)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    sys.exit("Erreur : impossible d'ouvrir la webcam.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    detection_result = detector.detect(mp_image)
    for detection in detection_result.detections:
        bbox = detection.bounding_box
        x_min = bbox.origin_x
        y_min = bbox.origin_y
        x_max = x_min + bbox.width
        y_max = y_min + bbox.height
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        roi = frame[y_min:y_max, x_min:x_max]
        if roi.size == 0:
            continue

        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        mp_roi = mp.Image(image_format=mp.ImageFormat.SRGB, data=roi_rgb)
        classification_result = classifier.classify(mp_roi)
        top_category = classification_result.classifications[0].categories[0]
        label = f"{top_category.category_name} ({top_category.score:.2f})"
        cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    cv2.imshow("Detection & Classification", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
